# Preprocessing

In [ ]:
import copy
import math
import random
from collections import OrderedDict, defaultdict

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
import torchvision.models as models
import torchvision
from torch.utils.data import DataLoader

from torchvision.datasets import *
from torchvision.transforms import *


no_cuda = False
use_gpu = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

In [ ]:
def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

class h_sigmoid(nn.Module):
    def __init__(self, inplace=True):
        super(h_sigmoid, self).__init__()
        self.relu = nn.ReLU6(inplace=inplace)

    def forward(self, x):
        return self.relu(x + 3) / 6


class h_swish(nn.Module):
    def __init__(self, inplace=True):
        super(h_swish, self).__init__()
        self.sigmoid = h_sigmoid(inplace=inplace)

    def forward(self, x):
        return x * self.sigmoid(x)

In [ ]:

class SELayer(nn.Module):
    def __init__(self, channel, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(channel, _make_divisible(channel // reduction, 8),bias=False),
                nn.ReLU(inplace=True),
                nn.Linear(_make_divisible(channel // reduction, 8), channel,bias=False),
                nn.ReLU(inplace=True),
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

batch_size = 32

#Dataset
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

#Dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Create NN model

In [ ]:
class ToyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.Conv = nn.Sequential(
      nn.Conv2d(in_channels=1, out_channels=5, kernel_size=1, stride=1,padding= 0, bias=True),
      h_swish(inplace=True),
      SELayer(5),
      nn.Conv2d(in_channels=5, out_channels=1, kernel_size=1, stride=1,padding= 0, bias=True),
      h_swish(inplace=True)
    )  
    self.backbone = nn.Sequential(
      nn.Linear(28*28, 120, bias=False),
      nn.ReLU(),
      nn.Linear(120, 84, bias=False),
      nn.ReLU(),
      nn.Linear(84, 10, bias=False)
    )

  def forward(self, x):
    x=self.Conv(x)
    x = x.view(-1, 28 * 28) #transform 28*28 figure to 784 vector
    x = self.backbone(x)
    return x

FP32_model = ToyModel()
print(FP32_model)

In [ ]:
#train model
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  #Set the model to train mode
  model.train()
  for batch, (x, y) in enumerate(dataloader):
    if use_gpu:
      x, y = x.cuda(), y.cuda()
    optimizer.zero_grad()
    #forward
    pred = model(x)

    #loss
    loss = loss_fn(pred, y)

    #backward
    loss.backward()

    #optimize
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(x)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  #set model to evaluate mode
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for x, y in dataloader:
      if use_gpu:
        x, y = x.cuda(), y.cuda()
      pred = model(x)
      test_loss = loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item() #calculate accuracy
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
learning_rate = 1e-3
epochs = 3
loss_fn = nn.CrossEntropyLoss() #define loss function
optimizer = torch.optim.Adam(FP32_model.parameters(), lr=learning_rate)  #define optimizer

FP32_model.to(device) #let model on GPU

In [87]:
#Training
for i in range(epochs):
  print(f"Epoch {i+1}\n-------------------------------")
  train_loop(train_loader, FP32_model, loss_fn, optimizer)
  test_loop(test_loader, FP32_model, loss_fn)

loss: 0.605188  [16032/60000]
loss: 0.421754  [19232/60000]
loss: 0.624769  [22432/60000]
loss: 0.629864  [25632/60000]
loss: 0.310080  [28832/60000]
loss: 0.612952  [32032/60000]
loss: 0.396862  [35232/60000]
loss: 0.431447  [38432/60000]
loss: 0.557648  [41632/60000]
loss: 0.559689  [44832/60000]
loss: 0.265188  [48032/60000]
loss: 0.728065  [51232/60000]
loss: 0.551859  [54432/60000]
loss: 0.174155  [57632/60000]
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.000908 

Epoch 2
-------------------------------
loss: 0.187655  [   32/60000]
loss: 0.264462  [ 3232/60000]
loss: 0.344568  [ 6432/60000]
loss: 0.388792  [ 9632/60000]
loss: 0.465516  [12832/60000]
loss: 0.222715  [16032/60000]
loss: 0.306143  [19232/60000]
loss: 0.681011  [22432/60000]
loss: 0.362475  [25632/60000]
loss: 0.335350  [28832/60000]
loss: 0.378258  [32032/60000]
loss: 0.502297  [35232/60000]
loss: 0.453380  [38432/60000]
loss: 0.406592  [41632/60000]
loss: 0.154795  [44832/60000]
loss: 0.325632  [48032/60000]
loss: 0.

# Quantization definition

####Question 1.####

Use
>$S=(r_{\mathrm{max}} - r_{\mathrm{min}}) / (q_{\mathrm{max}} - q_{\mathrm{min}})$

>$Z = q_{\mathrm{min}} - r_{\mathrm{min}} / S$

to calculate scale factor and zero point of a tensor


In [88]:
def get_scale_and_zero_point(fp32_tensor, bitwidth=8):
  q_min, q_max = -(2**(bitwidth-1)-1), 2**(bitwidth-1) - 1
  fp_min = fp32_tensor.min().item()
  fp_max = fp32_tensor.max().item()

  #####################################################

  scale = (fp_max-fp_min) / (q_max-q_min)
  zero_point = q_min-fp_min /scale

  #####################################################


  zero_point = round(zero_point)          #round
  zero_point = max(q_min, min(zero_point, q_max)) #clip

  return scale, int(zero_point)

####Question 2.####

Use $q=r/S + Z$ to quantize a tensor

In [89]:
def linear_quantize(fp32_tensor, bitwidth=8):
  q_min, q_max = -(2**(bitwidth-1)-1), 2**(bitwidth-1) - 1

  scale, zero_point = get_scale_and_zero_point(fp32_tensor)

  #####################################################

  q_tensor = torch.round( fp32_tensor/scale ) +zero_point

  #####################################################

  #clamp
  q_tensor = torch.clamp(q_tensor, q_min, q_max)
  return q_tensor, scale, zero_point  

####Question 3.####

Use
> $q_{\mathrm{output}} = M * \mathrm{Linear}[q_{\mathrm{input}}, q_{\mathrm{weight}}] + Z_{\mathrm{output}}$

> $M = S_{\mathrm{input}} * S_{\mathrm{weight}} / S_{\mathrm{output}}$

to compute quantized linear operation

In [90]:
def quantized_linear(input, weights, input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point, device, bitwidth=8, activation_bitwidth=8):
  input, weights = input.to(device), weights.to(device)

  #####################################################

  M = input_scale * weight_scale / output_scale
  output = torch.nn.functional.linear((input - input_zero_point ), (weights - weight_zero_point))
  output *= M
  output += output_zero_point

  #####################################################

  #clamp and round
  output = output.round().clamp(-(2**(activation_bitwidth-1)-1), 2**(activation_bitwidth-1)-1)

  return output

def quantized_conv(input, bias,weights,stride, padding,groups,input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point, device, bitwidth=8, activation_bitwidth=16):
  input, weights = input.to(device), weights.to(device)

  #####################################################

  M = input_scale * weight_scale 
  conv_bias = bias /M
  conv_bias = conv_bias.round()
  output_only_conv = torch.nn.functional.conv2d((input - input_zero_point ), (weights - weight_zero_point),bias = conv_bias ,stride=stride,padding=padding,groups=groups)
  output = M * output_only_conv
  #output += output_zero_point

  #####################################################

  #clamp and round
  output = output.round().clamp(-(2**(activation_bitwidth-1)-1), 2**(activation_bitwidth-1)-1)

  return output

def do_requant(input, scale,zero_point,bitwidth=8):
    output = input / scale
    output = output.round()
    output += zero_point
    output = output.clamp(-(2**(bitwidth-1)-1), 2**(bitwidth-1)-1)
    return output

def do_fake_quant(input, deq_scale, q_scale, q_zero_point,bitwidth=8):
    M = deq_scale/q_scale
    N = q_zero_point
    output = input * M
    output += N
    output = output.round().clamp(-(2**(bitwidth-1)-1), 2**(bitwidth-1)-1)
    return output
   

# Design quantized linear layer and preprocess

In [91]:
class Q_SELayer(nn.Module):
  def __init__(self,weights1,input_scale1,weight_scale1,output_scale1, input_zero_point1, weight_zero_point1, output_zero_point1,
               weights2, input_scale2, weight_scale2, output_scale2, input_zero_point2, weight_zero_point2, output_zero_point2,
               in_pool_scale,in_pool_zero_point,
               SE_req_q_scale,SE_req_q_output_zero_point,
               SE_req_dq_scale , SE_req_dq_output_zero_point):
    super().__init__()
    self.weights1 = weights1
    self.input_scale1 = input_scale1
    self.weight_scale1 = weight_scale1
    self.output_scale1 = output_scale1
    self.input_zero_point1 = input_zero_point1
    self.weight_zero_point1 = weight_zero_point1
    self.output_zero_point1 = output_zero_point1

    self.weights2 = weights2
    self.input_scale2 = input_scale2
    self.weight_scale2 = weight_scale2
    self.output_scale2 = output_scale2
    self.input_zero_point2 = input_zero_point2
    self.weight_zero_point2 = weight_zero_point2
    self.output_zero_point2 = output_zero_point2

    self.SE_req_dq_scale , self.SE_req_dq_output_zero_point = SE_req_dq_scale , SE_req_dq_output_zero_point

    self.SE_req_q_scale = SE_req_q_scale
    self.SE_req_q_output_zero_point = SE_req_q_output_zero_point

    self.in_pool_scale = in_pool_scale
    self.in_pool_zero_point = in_pool_zero_point

    self.avg_pool = nn.AdaptiveAvgPool2d(1)

  def forward(self,x):
    b, c, _, _ = x.size()
    y = self.avg_pool(x).view(b, c)
    y = (y-self.in_pool_zero_point) 
    y = do_fake_quant(y, self.in_pool_scale, self.input_scale1,self.input_zero_point1,bitwidth=8)
    y = quantized_linear(y, self.weights1, self.input_scale1, self.weight_scale1, self.output_scale1, self.input_zero_point1, self.weight_zero_point1, self.output_zero_point1, device)
    y = quantized_linear(y, self.weights2, self.input_scale2, self.weight_scale2, self.output_scale2, self.input_zero_point2, self.weight_zero_point2, self.output_zero_point2, device).view(b,c,1,1)
    z = (x-self.SE_req_dq_output_zero_point)*(y-self.output_zero_point2)
    return do_fake_quant(z, deq_scale=(self.SE_req_dq_scale *self.output_scale2), 
                         q_scale=self.SE_req_q_scale, q_zero_point=self.SE_req_q_output_zero_point,bitwidth=8)
  
  def __repr__(self):
    return f"Quantized_SE(in_channels={self.weights1.size(1)}, out_channels={self.weights2.size(0)})"
    


class QuantizedConv(nn.Module):
  def __init__(self,bias ,weights,stride,padding,groups ,input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point, bitwidth=8, activation_bitwidth=8):
    super().__init__()
    self.stride, self.padding, self.groups = stride, padding,groups
    self.weights = weights
    self.input_scale, self.input_zero_point = input_scale, input_zero_point
    self.weight_scale, self.weight_zero_point = weight_scale, weight_zero_point
    self.output_scale, self.output_zero_point = output_scale, output_zero_point
    self.bias = bias
    self.bitwidth = bitwidth
    self.activation_bitwidth = activation_bitwidth
    self.q_bias = torch.round(bias / (input_scale*weight_scale))
    self.q_weight = weights - weight_zero_point
    self.DeQ_scale = input_scale*weight_scale*8192
  def forward(self, x):
    return quantized_conv(x, self.bias, self.weights, self.stride, self.padding, self.groups, self.input_scale, self.weight_scale, self.output_scale, self.input_zero_point, self.weight_zero_point, self.output_zero_point, device)
  def __repr__(self):
    return f"QuantizedConv(in_channels={self.weights.size(1)}, out_channels={self.weights.size(0)})"

class QuantizedLinear(nn.Module):
  def __init__(self, weights, input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point, bitwidth=8, activation_bitwidth=8):
    super().__init__()
    self.weights = weights
    self.input_scale, self.input_zero_point = input_scale, input_zero_point
    self.weight_scale, self.weight_zero_point = weight_scale, weight_zero_point
    self.output_scale, self.output_zero_point = output_scale, output_zero_point

    self.bitwidth = bitwidth
    self.activation_bitwidth = activation_bitwidth

  def forward(self, x):
    return quantized_linear(x, self.weights, self.input_scale, self.weight_scale, self.output_scale, self.input_zero_point, self.weight_zero_point, self.output_zero_point, device)
  def __repr__(self):
    return f"QuantizedLinear(in_channels={self.weights.size(1)}, out_channels={self.weights.size(0)})"

#Transform input data to correct integer range
class Preprocess(nn.Module):
  def __init__(self, input_scale, input_zero_point, activation_bitwidth=8):
    super().__init__()
    self.input_scale, self.input_zero_point = input_scale, input_zero_point
    self.activation_bitwidth = activation_bitwidth
  def forward(self, x):
    x = x / self.input_scale + self.input_zero_point
    x = x.round() 
    return x
  
class Quantizer(nn.Module):
  def __init__(self,scale,zero_point,bitwidth=8):
    super().__init__()
    self.scale = scale
    self.zero = zero_point
    self.store_scale = scale *64

  def forward(self,x):
    return do_requant(x,self.scale,self.zero)
    

# Calibration

In [101]:
# add hook to record the min max value of the activation
input_activation = {}
output_activation = {}

#Define a hook to record the feature map of each layer
def add_range_recoder_hook(model):
    import functools
    def _record_range(self, x, y, module_name):
        x = x[0]
        input_activation[module_name] = x.detach()
        output_activation[module_name] = y.detach()

    all_hooks = []
    for name, m in model.named_modules():
        if isinstance(m, (nn.Linear, nn.ReLU,nn.Conv2d,h_swish,nn.AdaptiveAvgPool2d)):
            all_hooks.append(m.register_forward_hook(
                functools.partial(_record_range, module_name=name)))


    return all_hooks

hooks = add_range_recoder_hook(FP32_model)
sample_data = iter(train_loader).__next__()[0].to(device) #Use a batch of training data to calibrate
FP32_model(sample_data) #Forward to use hook
# print(output_activation['Conv.1'])
# print("==")
# print(input_activation['Conv.2.avg_pool'])
print(output_activation.keys())
# remove hooks
for h in hooks:
    h.remove()


dict_keys(['Conv.0', 'Conv.1', 'Conv.2.avg_pool', 'Conv.2.fc.0', 'Conv.2.fc.1', 'Conv.2.fc.2', 'Conv.2.fc.3', 'Conv.3', 'Conv.4', 'backbone.0', 'backbone.1', 'backbone.2', 'backbone.3', 'backbone.4'])


# Quantize model

In [93]:
#copy original model
quantized_model = copy.deepcopy(FP32_model)

#Record each layer in original model
quantized_backbone = []
quantized_Conv = []
i = 0

#Record input scale and zero point
input_scale, input_zero_point = get_scale_and_zero_point(input_activation["Conv.0"])
preprocess = Preprocess(input_scale, input_zero_point)
quantized_Conv.append(preprocess)

input_scale, input_zero_point = get_scale_and_zero_point(input_activation['Conv.0'])
output_scale, output_zero_point = get_scale_and_zero_point(output_activation['Conv.0'])
quantized_weights, weight_scale, weight_zero_point = linear_quantize(FP32_model.Conv[0].weight.data)
Conv_bias = FP32_model.Conv[0].bias.data
quantizedConv1 = QuantizedConv(Conv_bias,quantized_weights, 1,0,1,input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point)
h_swish1 = h_swish()
#quantized_model.Conv[0] = quantizedConv1 

req_scale , output_zero_point = get_scale_and_zero_point(output_activation['Conv.1'])
req1 = Quantizer(req_scale,output_zero_point)

quantized_Conv.append(quantizedConv1)
quantized_Conv.append(h_swish1)
quantized_Conv.append(req1)
###############
input_scale1, input_zero_point1 = get_scale_and_zero_point(input_activation['Conv.2.fc.0'])
output_scale1, output_zero_point1 = get_scale_and_zero_point(output_activation['Conv.2.fc.1'])
quantized_weights1, weight_scale1, weight_zero_point1 = linear_quantize(FP32_model.Conv[2].fc[0].weight.data)

input_scale2, input_zero_point2 = get_scale_and_zero_point(input_activation['Conv.2.fc.2'])
output_scale2, output_zero_point2 = get_scale_and_zero_point(output_activation['Conv.2.fc.3'])
quantized_weights2, weight_scale2, weight_zero_point2 = linear_quantize(FP32_model.Conv[2].fc[2].weight.data)

SE_in_pool_scale, SE_in_pool_zero_point = get_scale_and_zero_point(input_activation['Conv.2.avg_pool'])

SE_req_q_scale , SE_req_q_output_zero_point = get_scale_and_zero_point(input_activation['Conv.3'])

SE_req_dq_scale , SE_req_dq_output_zero_point = get_scale_and_zero_point(output_activation['Conv.1'])

quantizedSE_linear1 =Q_SELayer(quantized_weights1,input_scale1,weight_scale1,output_scale1,input_zero_point1,weight_zero_point1,output_zero_point1, 
                               quantized_weights2,input_scale2,weight_scale2,output_scale2,input_zero_point2,weight_zero_point2,output_zero_point2,
                               SE_in_pool_scale, SE_in_pool_zero_point,
                               SE_req_q_scale,SE_req_q_output_zero_point,
                               SE_req_dq_scale , SE_req_dq_output_zero_point)
##################
quantized_Conv.append(quantizedSE_linear1)


input_scale, input_zero_point = get_scale_and_zero_point(input_activation['Conv.3'])
output_scale, output_zero_point = get_scale_and_zero_point(output_activation['Conv.3'])
quantized_weights, weight_scale, weight_zero_point = linear_quantize(FP32_model.Conv[3].weight.data)
Conv_bias = FP32_model.Conv[3].bias.data
quantizedConv2 = QuantizedConv(Conv_bias,quantized_weights, 1,0,1,input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point)
#quantized_model.Conv[2] = quantizedConv2 

h_swish2 = h_swish()
#quantized_model.Conv[0] = quantizedConv1 

req_scale , output_zero_point = get_scale_and_zero_point(output_activation['Conv.4'])
req2 = Quantizer(req_scale,output_zero_point)

quantized_Conv.append(quantizedConv2)
quantized_Conv.append(h_swish2)
quantized_Conv.append(req2)


################# below is linear

input_scale, input_zero_point = get_scale_and_zero_point(input_activation['backbone.0'])
output_scale, output_zero_point = get_scale_and_zero_point(output_activation['backbone.1'])
quantized_weights, weight_scale, weight_zero_point = linear_quantize(FP32_model.backbone[0].weight.data)
quantizedLinear1 = QuantizedLinear(quantized_weights, input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point)
quantized_backbone.append(quantizedLinear1)

input_scale, input_zero_point = get_scale_and_zero_point(input_activation['backbone.2'])
output_scale, output_zero_point = get_scale_and_zero_point(output_activation['backbone.3'])
quantized_weights, weight_scale, weight_zero_point = linear_quantize(FP32_model.backbone[2].weight.data)
quantizedLinear2 = QuantizedLinear(quantized_weights, input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point)
quantized_backbone.append(quantizedLinear2)

# #Record Linear + ReLU of the model (except the last Linear)
# while i < len(quantized_model.backbone) - 1:
#   if isinstance(quantized_model.backbone[i], nn.Linear) and isinstance(quantized_model.backbone[i+1], nn.ReLU):
#     linear = quantized_model.backbone[i]
#     linear_name = f"backbone.{i}"
#     relu = quantized_model.backbone[i + 1]
#     relu_name = f"backbone.{i + 1}"

#     #Use the calibration data to calculate scale and zero point of each layer
#     input_scale, input_zero_point = get_scale_and_zero_point(input_activation[linear_name])
#     output_scale, output_zero_point = get_scale_and_zero_point(output_activation[relu_name])
#     quantized_weights, weight_scale, weight_zero_point = linear_quantize(linear.weight.data)

#     quantizedLinear = QuantizedLinear(quantized_weights, input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point)

#     quantized_backbone.append(quantizedLinear)
#     i += 2

#Record the last Linear layer
linear = quantized_model.backbone[4]
linear_name = f"backbone.4"
input_scale, input_zero_point = get_scale_and_zero_point(input_activation[linear_name])
output_scale, output_zero_point = get_scale_and_zero_point(output_activation[linear_name])
quantized_weights, weight_scale, weight_zero_point = linear_quantize(linear.weight.data)
quantizedLinear3 = QuantizedLinear(quantized_weights, input_scale, weight_scale, output_scale, input_zero_point, weight_zero_point, output_zero_point)
quantized_backbone.append(quantizedLinear3)

quantized_model.Conv = nn.Sequential(*quantized_Conv)
quantized_model.backbone = nn.Sequential(*quantized_backbone)

In [94]:
print(quantized_model)

ToyModel(
  (Conv): Sequential(
    (0): Preprocess()
    (1): QuantizedConv(in_channels=1, out_channels=5)
    (2): h_swish(
      (sigmoid): h_sigmoid(
        (relu): ReLU6(inplace=True)
      )
    )
    (3): Quantizer()
    (4): Quantized_SE(in_channels=5, out_channels=5)
    (5): QuantizedConv(in_channels=5, out_channels=1)
    (6): h_swish(
      (sigmoid): h_sigmoid(
        (relu): ReLU6(inplace=True)
      )
    )
    (7): Quantizer()
  )
  (backbone): Sequential(
    (0): QuantizedLinear(in_channels=784, out_channels=120)
    (1): QuantizedLinear(in_channels=120, out_channels=84)
    (2): QuantizedLinear(in_channels=84, out_channels=10)
  )
)


In [95]:
# add hook to record the min max value of the activation
q_input_activation = {}
q_output_activation = {}

#Define a hook to record the feature map of each layer
def add_range_recoder_hook(model):
    import functools
    def _record_range(self, x, y, module_name):
        x = x[0]
        q_input_activation[module_name] = x.detach()
        q_output_activation[module_name] = y.detach()

    all_hooks = []
    for name, m in model.named_modules():
        if isinstance(m, (QuantizedConv,  QuantizedLinear,h_swish,Quantizer,Preprocess,Q_SELayer)):
            all_hooks.append(m.register_forward_hook(
                functools.partial(_record_range, module_name=name)))


    return all_hooks


q_test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)
hooks = add_range_recoder_hook(quantized_model)
sample_data = iter(q_test_loader).__next__()[0].to(device) #Use a batch of training data to calibrate
quantized_model(sample_data) #Forward to use hook

print(q_output_activation.keys())
#print(q_intput_activation.keys())
print(quantized_model.Conv[4].weights2)
print(q_output_activation["Conv.4"])
# remove hooks
for h in hooks:
    h.remove()

dict_keys(['Conv.0', 'Conv.1', 'Conv.2', 'Conv.3', 'Conv.4', 'Conv.5', 'Conv.6', 'Conv.7', 'backbone.0', 'backbone.1', 'backbone.2'])
tensor([[ -78.,   40.,  -74.,   43.,  -75.,  -61.,   36., -113.],
        [  10.,   -3.,  -71., -127.,   85.,   12.,   -4.,  -69.],
        [ 114.,  -19.,   53.,  102.,  127.,  -25.,  -69.,  105.],
        [-103.,   51.,    0.,   51., -111., -113.,  -62.,  -70.],
        [  70.,    9.,   16.,   42.,   51.,   59.,  -51.,  101.]],
       device='cuda:0')
tensor([[[[-81., -81., -81.,  ..., -81., -81., -81.],
          [-81., -81., -81.,  ..., -81., -81., -81.],
          [-81., -81., -81.,  ..., -81., -81., -81.],
          ...,
          [-81., -81., -81.,  ..., -81., -81., -81.],
          [-81., -81., -81.,  ..., -81., -81., -81.],
          [-81., -81., -81.,  ..., -81., -81., -81.]],

         [[-98., -98., -98.,  ..., -98., -98., -98.],
          [-98., -98., -98.,  ..., -98., -98., -98.],
          [-98., -98., -98.,  ..., -98., -98., -98.],
        

In [96]:


# print(q_input_activation["Conv.0"].shape)
print(q_input_activation["Conv.1"].shape)
# print(q_input_activation["Conv.2"].shape)

for i in range(28):
    print(q_output_activation["Conv.1"][0][0][0])
#print(q_input_activation["Conv.1"][0][0][1])

torch.Size([1, 1, 28, 28])
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       device='cuda:0')
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       device='cuda:0')
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       device='cuda:0')
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       device='cuda:0')
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       device='cuda:0')
tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., 

In [97]:
def signed_dec2hex_matrix(input):
    '''Convert a matrix which data is signed decimal to 8 bits hex with 2's complement'''
    temp = []
    bin8 = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(8)] ) )
    for i in input:
        test =bin8(i)
        test = int(test,base=2)
        hex_test = hex(test)[2:].zfill(2)
        temp.append(hex_test)

    return temp

def signed_dec2hex(input):
    '''Convert a number which data is signed decimal to 8 bits hex with 2's complement'''
    bin8 = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(8)] ) )
    test =bin8(input)
    test = int(test,base=2)
    hex_test = hex(test)[2:].zfill(2)

    return hex_test


def golden_gen(golden_layer_decimal):
    '''Convert a layer output which is signed decimal in GPU to 8 bits hex with 2's complement in CPU and
     make it 4 element in a line, example of use : golden_gen(q_output_activation["Conv.3"]) '''
    golden = []
    i=0
    golden_in_numpy = golden_layer_decimal.cpu().numpy()
    test = golden_in_numpy.flatten()
    test =test.astype('int32')
    golden.append([])
    for j, data in enumerate(test):
        if(j%4==0 ):
            golden.append([])
            i = i+1
            golden[i].append(signed_dec2hex(data))
        if(j%4!=0):
            golden[i].append(signed_dec2hex(data))
    golden.pop(0)
    for indice,data in enumerate(golden):
        print(*data,sep='')

def input_or_weight_gen(layer_decimal):
    '''Convert a layer output which is signed decimal in GPU to 8 bits hex with 2's complement in CPU and
     make each byte display in different place, example of use : input_or_weight_gen(quantized_model.Conv[1].weights)'''
    byte0 = []
    byte1 = []
    byte2 = []
    byte3 = []

    data_in_numpy = layer_decimal.cpu().numpy()
    data_test = data_in_numpy.flatten()
    data_test = data_test.astype('int32')
    data_test = signed_dec2hex_matrix(data_test)
    for indice,data in enumerate(data_test):
        if(indice%4 == 0):
            byte0.append(data)
        elif(indice%4 == 1):
            byte1.append(data)
        elif(indice%4 == 2):
            byte2.append(data)
        else:
            byte3.append(data)
    print("byte0:",*byte0)
    print("=======")
    print("byte1:",*byte1)
    print("=======")
    print("byte2:",*byte2)
    print("=======")
    print("byte3:",*byte3)
    print("=======")
    return byte0,byte1,byte2,byte3

def DecToBin_machine(num,accuracy):
    integer = int(num)
    flo = num - integer
    integercom = '{:1b}'.format(integer)
    tem = flo
    flo_list = []
    for i in range(accuracy):
        tem *= 2
        flo_list += str(int(tem))
        tem -= int(tem)
    flocom = flo_list
    binary_value =  ''.join(flocom)
    return binary_value

#golden_gen(q_output_activation["Conv.3"])
print(signed_dec2hex(quantized_model.Conv[1].input_zero_point))
input_or_weight_gen(q_input_activation["Conv.1"])
print("===")
input_or_weight_gen(quantized_model.Conv[4].weights)

print(quantized_model.Conv[4].input_zero_point)

DeS = quantized_model.Conv[3].store_scale
print(DeS)
#print(DeS *8192) # 2**13
#print("deq_scale (shift 13):",DecToBin_machine(quantized_model.Conv[1].,8))
print("req_scale (shift 6):",)
print("output zero",)
#binary_value = integercom + '.' + ''.join(flocom)
result = DecToBin_machine(DeS,8)
print(result)
# 0.1100111101011100

00
byte0: 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 82 81 81 81 81 81 9c 81 81 81 81 81 81 d9 81 de 81 81 81 81 01 1d 28 81 81 81 83 02 1f 17 81 81 83 81 eb 1c 1d 81 82 81 81 df 17 1f 81 81 82 e3 ef 15 1e 81 82 81 f6 08 1a 13 84 81 b6 e8 0f 1a 0a 81 c2 00 f7 05 1b 2c 81 f0 f4 0f 1a 22 19 c7 e3 e3 f8 1c 44 45 91 3c 07 12 50 75 77 81 c4 47 50 89 32 22 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81
byte1: 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 81 a6 81 81 81 82 d5 81 f8 81 81 81 82 0f 81 eb 81 81 81 85 f9 26 0c 81 81 81 81 00 27 10 81 81 82 81 09 25 1e 81 81 81 da 19 29 29 81 81 81 08 22 1f 29 81 83 81 e4 1c 1c 1c 81 81 dd f4 1a 17 16 81 cd fc 00 08 21 21 c5 f3 00 ff 0f 22

AttributeError: 'Q_SELayer' object has no attribute 'weights'

# Evaluate

In [98]:
test_loop(test_loader, FP32_model, loss_fn)

Test Error: 
 Accuracy: 87.0%, Avg loss: 0.000476 



In [100]:
test_loop(test_loader, quantized_model, loss_fn)

Test Error: 
 Accuracy: 32.4%, Avg loss: 0.014689 

